## 資料載入

In [ ]:
import pandas as pd

In [ ]:
# 讀入 csv 文字檔
csv_file = "https://storage.googleapis.com/learn_pd_like_tidyverse/gapminder.csv"
gapminder = pd.read_csv(csv_file)
print(type(gapminder))
gapminder.head()

In [ ]:
# 讀入 excel 試算表
xlsx_file = "https://storage.googleapis.com/learn_pd_like_tidyverse/gapminder.xlsx"
gapminder = pd.read_excel(xlsx_file)
print(type(gapminder))
gapminder.head()

In [ ]:
gapminder.shape

In [ ]:
gapminder.columns

In [ ]:
gapminder.index

In [ ]:
gapminder.info()

In [ ]:
gapminder.describe()

## 資料整理

In [ ]:
gapminder[gapminder['country'] == 'Taiwan']

In [ ]:
gapminder[(gapminder['year'] == 2007) & (gapminder['continent'] == 'Asia')]

In [ ]:
gapminder[['country', 'continent']]

In [ ]:
country = gapminder['country']
type(country)

In [ ]:
gapminder['country_abb'] = gapminder['country'].apply(lambda x: x[:3])
gapminder

In [ ]:
gapminder[gapminder['year'] == 2007][['pop']].sum()

In [ ]:
gapminder[gapminder['year'] == 2007][['lifeExp', 'gdpPercap']].mean()

In [ ]:
gapminder[gapminder['year'] == 2007].groupby(by = 'continent')['pop'].sum()

In [ ]:
gapminder[gapminder['year'] == 2007].groupby(by = 'continent')[['lifeExp', 'gdpPercap']].mean()

## 資料視覺化

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
gapminder_twn = gapminder[gapminder['country'] == 'Taiwan']
gapminder_twn[['year', 'pop']].plot(kind = 'line', x = 'year', y = 'pop', title = 'Pop vs. Year in Taiwan', legend = False)
plt.show()

In [ ]:
gapminder_northasia = gapminder.loc[gapminder['country'].isin(['China', 'Japan', 'Korea, Rep.', 'Taiwan'])]
gapminder_northasia_pivot = gapminder_northasia.pivot_table(values = 'lifeExp', columns = 'country', index = 'year')
gapminder_northasia_pivot.plot(title = 'Life Expectancies in North Asia')
plt.show()

In [ ]:
gapminder_2007 = gapminder[gapminder['year'] == 2007]
gapminder_2007[['pop', 'gdpPercap', 'lifeExp']].hist(bins = 15)
plt.show()

In [ ]:
gapminder_2007[['gdpPercap']].plot(kind = 'hist', title = 'GDP Per Capita in 2007', legend = False, bins = 15)
plt.show()

In [ ]:
gapminder_continent_pivot = gapminder_2007.pivot_table(values = 'gdpPercap', columns = 'continent', index = 'country')
gapminder_continent_pivot.plot(kind = 'hist', alpha=0.5, bins = 20, title = 'GDP Per Capita by Continent')
plt.show()

In [ ]:
gapminder_continent_pivot.plot(kind = 'box', title = 'GDP Per Capita by Continent')
plt.show()

In [ ]:
gapminder_2007.plot(kind = 'scatter', x = 'gdpPercap', y = 'lifeExp', title = 'Wealth vs. Health in 2007')
plt.show()

In [ ]:
gapminder_2007.plot(kind = 'hexbin', x = 'gdpPercap', y = 'lifeExp', title = 'Wealth vs. Health in 2007', gridsize = 20)
plt.show()

In [ ]:
summarized_df = gapminder[gapminder['year'] == 2007].groupby(by = 'continent')['pop'].sum()
summarized_df.plot(kind = 'bar', rot = 0)
plt.show()

In [ ]:
summarized_df = gapminder[gapminder['year'] == 2007].groupby(by = 'continent')[['lifeExp', 'gdpPercap']].mean()
summarized_df.plot(kind = 'barh', subplots = True, layout = (1, 2), sharex = False, sharey = True, legend = False)
plt.show()

## Series 與 Panel

In [ ]:
country = gapminder['country']
print(type(country))

In [ ]:
print(country.values)
print(type(country.values))

In [ ]:
df_grouped = gapminder.groupby(['year'])
df_dict = {}
for i in range(1952, 2011, 5):
    df_dict[i] = df_grouped.get_group(i).reset_index(drop = True)
gapminder_panel = pd.Panel(df_dict)
gapminder_panel

## 練習：整理 gapminder.org 的資料

In [ ]:
def get_data(url_list):
    df_list = []
    for url in url_list:
        df_list.append(pd.read_excel(url, sheetname = 'Data'))
    return df_list

In [ ]:
url_list = ['https://storage.googleapis.com/learn_pd_like_tidyverse/indicator_gapminder_population.xlsx', 'https://storage.googleapis.com/learn_pd_like_tidyverse/indicator_gapminder_gdp_per_capita_ppp.xlsx', 'https://storage.googleapis.com/learn_pd_like_tidyverse/indicator_life_expectancy_at_birth.xlsx']
wide_df_list = get_data(url_list)

In [ ]:
wide_df_list[0].head()

In [ ]:
wide_df_list[1].head()

In [ ]:
wide_df_list[2].head()

In [ ]:
def get_long_df(wide_df_list):
    long_df_list = []
    source_var = ['Total population', 'GDP per capita', 'Life expectancy']
    renamed_var = ['pop', 'gdpPercap', 'lifeExp']
    for (i, old_var, new_var) in zip(range(3), source_var, renamed_var):
        df = pd.melt(wide_df_list[i], id_vars = [old_var])
        df.columns = ['country', 'year', new_var]
        long_df_list.append(df)
    return long_df_list

In [ ]:
long_df_list = get_long_df(wide_df_list)

In [ ]:
long_df_list[0].head()

In [ ]:
long_df_list[1].head()

In [ ]:
long_df_list[2].head()

In [ ]:
merged_df = pd.merge(long_df_list[0], long_df_list[1], on = ['country', 'year'])
merged_df = pd.merge(merged_df, long_df_list[2], on = ['country', 'year'])
merged_df = merged_df.dropna()
merged_df = merged_df.sort_values(['year', 'country'])
merged_df = merged_df.reset_index(drop = True)
merged_df.head()

In [ ]:
merged_df.to_csv('gapminder.csv', index = False)
merged_df.to_excel('gapminder.xlsx', index = False)